# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install Cartopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geoviews

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
all_city_data_df = pd.read_csv("cities.csv")

# Display sample data
all_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sardarshahr,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
1,1,Torghay,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
2,2,Saipan,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
3,3,Antalaha,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
4,4,Grytviken,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
humidity = all_city_data_df.hvplot.points('Lng', 'Lat', geo=True, title='OSM', color='City', s='Humidity')

# Display the map
humidity

:Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = all_city_data_df.loc[(all_city_data_df['Max Temp'] >70) & (all_city_data_df['Humidity'] < 50) & 
                           (all_city_data_df['Wind Speed'] < 6),:]
len(ideal_weather)

# Drop any rows with null values
ideal_weather = all_city_data_df.dropna()

# Display sample data
ideal_weather_df = ideal_weather.sort_values(by=['Cloudiness'])
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Sardarshahr,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
387,387,Siwa Oasis,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
386,386,Matane,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
385,385,Baotou,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
384,384,Carauari,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
...,...,...,...,...,...,...,...,...,...,...
190,190,Bonthe,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
191,191,Sejenane,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
192,192,Puerto San Carlos,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393
186,186,Fort Mcmurray,48.2553,72.8483,-3.57,100,45,2.28,KZ,1706460393


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Sardarshahr,KZ,48.2553,72.8483,100,
387,Siwa Oasis,KZ,48.2553,72.8483,100,
386,Matane,KZ,48.2553,72.8483,100,
385,Baotou,KZ,48.2553,72.8483,100,
384,Carauari,KZ,48.2553,72.8483,100,
...,...,...,...,...,...,...
190,Bonthe,KZ,48.2553,72.8483,100,
191,Sejenane,KZ,48.2553,72.8483,100,
192,Puerto San Carlos,KZ,48.2553,72.8483,100,
186,Fort Mcmurray,KZ,48.2553,72.8483,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
params =  {"radius": 10000,
           "types": "lodging",
            "apiKey": geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places" 

    # Make and API request using the params dictionaty
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Retrieving Results for Index 0: Sardarshahr.
Sardarshahr - nearest hotel: No hotel found
Retrieving Results for Index 387: Siwa Oasis.
Siwa Oasis - nearest hotel: No hotel found
Retrieving Results for Index 386: Matane.
Matane - nearest hotel: No hotel found
Retrieving Results for Index 385: Baotou.
Baotou - nearest hotel: No hotel found
Retrieving Results for Index 384: Carauari.
Carauari - nearest hotel: No hotel found
Retrieving Results for Index 383: Prado.
Prado - nearest hotel: No hotel found
Retrieving Results for Index 382: Pointe-Du-Lac.
Pointe-Du-Lac - nearest hotel: No hotel found
Retrieving Results for Index 381: Gorontalo.
Gorontalo - nearest hotel: No hotel found
Retrieving Results for Index 380: Sola.
Sola - nearest hotel: No hotel found
Retrieving Results for Index 379: Kudahuvadhoo.
Kudahuvadhoo - nearest hotel: No hotel found
Retrieving Results for Index 378: Iranduba.
Iranduba - nearest hotel: No hotel found
Retrieving Results for Index 377: Kyr

Puerto Deseado - nearest hotel: No hotel found
Retrieving Results for Index 292: Mareeba.
Mareeba - nearest hotel: No hotel found
Retrieving Results for Index 294: Vorkuta.
Vorkuta - nearest hotel: No hotel found
Retrieving Results for Index 295: Solleftea.
Solleftea - nearest hotel: No hotel found
Retrieving Results for Index 296: Medvezh'Yegorsk.
Medvezh'Yegorsk - nearest hotel: No hotel found
Retrieving Results for Index 297: Vingt Cinq.
Vingt Cinq - nearest hotel: No hotel found
Retrieving Results for Index 298: Howard Springs.
Howard Springs - nearest hotel: No hotel found
Retrieving Results for Index 299: Al Mithnab.
Al Mithnab - nearest hotel: No hotel found
Retrieving Results for Index 293: Puerto Armuelles.
Puerto Armuelles - nearest hotel: No hotel found
Retrieving Results for Index 318: Robore.
Robore - nearest hotel: No hotel found
Retrieving Results for Index 319: Carbonear.
Carbonear - nearest hotel: No hotel found
Retrieving Results for Index 320: Manzini.
Manzini - near

Shizunai-Furukawacho - nearest hotel: No hotel found
Retrieving Results for Index 564: Cromer.
Cromer - nearest hotel: No hotel found
Retrieving Results for Index 565: Cuenca.
Cuenca - nearest hotel: No hotel found
Retrieving Results for Index 566: Nogliki.
Nogliki - nearest hotel: No hotel found
Retrieving Results for Index 560: Ermelo.
Ermelo - nearest hotel: No hotel found
Retrieving Results for Index 499: Ban Ratsada.
Ban Ratsada - nearest hotel: No hotel found
Retrieving Results for Index 551: Al Jawf.
Al Jawf - nearest hotel: No hotel found
Retrieving Results for Index 549: Kalymnos.
Kalymnos - nearest hotel: No hotel found
Retrieving Results for Index 535: Viet Tri.
Viet Tri - nearest hotel: No hotel found
Retrieving Results for Index 536: Togur.
Togur - nearest hotel: No hotel found
Retrieving Results for Index 537: Da Nang.
Da Nang - nearest hotel: No hotel found
Retrieving Results for Index 538: San Luis De La Loma.
San Luis De La Loma - nearest hotel: No hotel found
Retrievi

Chelno-Vershiny - nearest hotel: No hotel found
Retrieving Results for Index 284: Lhokseumawe.
Lhokseumawe - nearest hotel: No hotel found
Retrieving Results for Index 282: Copiapo.
Copiapo - nearest hotel: No hotel found
Retrieving Results for Index 102: Hasaki.
Hasaki - nearest hotel: No hotel found
Retrieving Results for Index 101: Papatowai.
Papatowai - nearest hotel: No hotel found
Retrieving Results for Index 100: West Island.
West Island - nearest hotel: No hotel found
Retrieving Results for Index 99: Pierre.
Pierre - nearest hotel: No hotel found
Retrieving Results for Index 98: Isafjordur.
Isafjordur - nearest hotel: No hotel found
Retrieving Results for Index 97: Sitka.
Sitka - nearest hotel: No hotel found
Retrieving Results for Index 96: Shitanjing.
Shitanjing - nearest hotel: No hotel found
Retrieving Results for Index 95: Tingo Maria.
Tingo Maria - nearest hotel: No hotel found
Retrieving Results for Index 94: Argo.
Argo - nearest hotel: No hotel found
Retrieving Results 

Utrik - nearest hotel: No hotel found
Retrieving Results for Index 32: Albany.
Albany - nearest hotel: No hotel found
Retrieving Results for Index 7: Magadan.
Magadan - nearest hotel: No hotel found
Retrieving Results for Index 9: Litochoro.
Litochoro - nearest hotel: No hotel found
Retrieving Results for Index 10: Coquimbo.
Coquimbo - nearest hotel: No hotel found
Retrieving Results for Index 11: Bilibino.
Bilibino - nearest hotel: No hotel found
Retrieving Results for Index 12: Blackmans Bay.
Blackmans Bay - nearest hotel: No hotel found
Retrieving Results for Index 13: Avarua.
Avarua - nearest hotel: No hotel found
Retrieving Results for Index 14: Vorgashor.
Vorgashor - nearest hotel: No hotel found
Retrieving Results for Index 8: Tiksi.
Tiksi - nearest hotel: No hotel found
Retrieving Results for Index 33: Puerto Natales.
Puerto Natales - nearest hotel: No hotel found
Retrieving Results for Index 34: Weno.
Weno - nearest hotel: No hotel found
Retrieving Results for Index 35: Coloni

Cockburn Town - nearest hotel: No hotel found
Retrieving Results for Index 280: Mutsu.
Mutsu - nearest hotel: No hotel found
Retrieving Results for Index 281: Detroit Lakes.
Detroit Lakes - nearest hotel: No hotel found
Retrieving Results for Index 275: Lerwick.
Lerwick - nearest hotel: No hotel found
Retrieving Results for Index 214: Hihifo.
Hihifo - nearest hotel: No hotel found
Retrieving Results for Index 266: Nadi.
Nadi - nearest hotel: No hotel found
Retrieving Results for Index 264: Salym.
Salym - nearest hotel: No hotel found
Retrieving Results for Index 250: Narrabri.
Narrabri - nearest hotel: No hotel found
Retrieving Results for Index 251: Katsuren-Haebaru.
Katsuren-Haebaru - nearest hotel: No hotel found
Retrieving Results for Index 252: Nagqu.
Nagqu - nearest hotel: No hotel found
Retrieving Results for Index 253: Borisoglebsk.
Borisoglebsk - nearest hotel: No hotel found
Retrieving Results for Index 254: Kalemie.
Kalemie - nearest hotel: No hotel found
Retrieving Results 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,Sardarshahr,KZ,48.2553,72.8483,100,No hotel found
387,Siwa Oasis,KZ,48.2553,72.8483,100,No hotel found
386,Matane,KZ,48.2553,72.8483,100,No hotel found
385,Baotou,KZ,48.2553,72.8483,100,No hotel found
384,Carauari,KZ,48.2553,72.8483,100,No hotel found
...,...,...,...,...,...,...
190,Bonthe,KZ,48.2553,72.8483,100,No hotel found
191,Sejenane,KZ,48.2553,72.8483,100,No hotel found
192,Puerto San Carlos,KZ,48.2553,72.8483,100,No hotel found
186,Fort Mcmurray,KZ,48.2553,72.8483,100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
final_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, title='OSM', color='City', hover=True, hover_cols='all',
                                   use_index=False, s='Humidity')

# Display the map
final_plot

:Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)